In [1]:
import polars as pl
import plotly.express as px

In [2]:
results = pl.read_csv("72_core_benchmark_result.csv")
results.head()

library,mode,workers,execution time
str,str,i64,f64
"""polars""","""lazy""",1,5.202698
"""polars""","""eager""",1,24.478623
"""duckdb""",null,1,4.262475
"""polars""","""lazy""",2,2.970445
"""polars""","""eager""",2,16.097312


In [12]:
def single_benchmark_plot(title, df):
    df = df.with_columns(pl.when(pl.col("execution time")==pl.col("execution time").min()).then("orange").otherwise("blue").alias("color"))
    fig = px.bar(
        x=df["workers"],
        y=df["execution time"],
        color=df["color"],
        color_discrete_map={
            "orange": "rgb(239,85,59)",
            "blue": "rgb(99,110,250)"
        },
        title=title,
        labels={'x': 'workers', 'y':'execution time (s)'},
        template="plotly_dark"
    )
    fig.add_hline(
        y=df["execution time"].min(),
        line_width=2,
        line_dash="dash",
        line_color="rgb(239,85,59)",
    )
    fig.update_traces(showlegend=False)
    return(fig)

In [13]:
single_benchmark_plot(
    "Polars lazy mode",
    results.filter((pl.col("library")=="polars") & (pl.col("mode")=="lazy"))
)

In [14]:
single_benchmark_plot(
    "Polars eager mode",
    results.filter((pl.col("library")=="polars") & (pl.col("mode")=="eager"))
)

In [15]:
single_benchmark_plot(
    "DuckDB",
    results.filter((pl.col("library")=="duckdb"))
)